# 21 - Weather to Module Performance
## Modeling Performance, an End to End Simulation

This tutorial shows how to use the new function on bifacial_radiance calculatePerformanceModule performance, as well as how to find CEC Module parameters.


In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_21')

if not os.path.exists(testfolder): os.mkdir(testfolder)

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_21


In [2]:
import bifacial_radiance
import numpy as np
import pandas as pd
import pvlib 

bifacial_radiance.__version__

'0.4.0+3.g7bc546a.dirty'

In [3]:
# Selecting only two times as examples
starttime = '01_13_11';  endtime = '01_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
demo.setGround(0.2)
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9) 
sceneDict = {'tilt': 10, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':3, 'nRows': 2}
trackerdict = demo.set1axis(metdata = metdata, cumulativesky = False)
trackerdict = demo.gendaylit1axis()
trackerdict = demo.makeScene1axis(moduletype = mymodule, sceneDict = sceneDict)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(sensorsy=3)


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct
Linescan in pro

## Geting a CEC Module

In [4]:
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.


Find the module that you want. In this case we know it's a Canadian of model CS6K-275M. 

Make sure you select only 1 module from the database -- sometimes there are similar names.

In [5]:
modfilter2 = db.index.str.startswith('Canadian') & db.index.str.endswith('CS6K-275M')
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

1  modules selected. Name of 1st entry:  Canadian Solar Inc. CS6K-275M


## Calculating the Performance and Exporting the Results to a CSV

In [6]:
demo.calculatePerformanceModule(CECMod=CECMod)

Bifaciality factor of module stored is  0.9


{'2021-01-13_1100': {'surf_azm': 90.0,
  'surf_tilt': 44.14,
  'theta': -44.14,
  'ghi': 211,
  'dhi': 149,
  'temp_air': 4.6,
  'wind_speed': 3.8,
  'skyfile': 'skies\\sky2_37.5_-77.33_2021-01-13_1100.rad',
  'clearance_height': 0.8035860263044873,
  'radfile': 'objects\\1axis2021-01-13_1100__C_0.80359_rtr_5.00000_tilt_44.14000_3modsx2rows_origin0,0.rad',
  'scene': {'module': {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.01, 'sceney': 2.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 0.9, 'text': '! genbox black test-module 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0', 'modulefile': 'objects\\test-module.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects\\test-module.rad', 'hpc': False, 'gcr': 0.4, 'text': '!xform -rx 44.14 -t 0 0 1.5 -a 3 -t 1.01 0 0 -a 2 -t 0 5 0 -i 1 -t -1.01 -0.0 0 -rz 90.0 -t 0 0 0 objects\\test-module.rad', 'radfiles': 'objects\\1axis2021-01-13_1100__C_0.80359_rtr_5.00000_tilt_44.14000_3

In [7]:
demo.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict


,Unnamed: 0,dhi,ghi,Wm2Back,Wm2Front,theta,surf_tilt,surf_azm,clearance_height,effective_irradiance,Pout_module,Wm2BackAvg,Wm2FrontAvg,BifiRatio
0,2021-01-13_1100,149,211,"[44.73869, 44.99094, 45.69043]","[254.3149, 255.1848, 253.8197]",-44.14,44.14,90.0,0.803586,295.065818,85.508565,45.14002,254.439800,0.159668
1,2021-01-13_1200,200,249,"[43.23021, 43.13953999999999, 43.9237]","[252.81730000000002, 252.9561, 253.0891]",-21.20,21.20,90.0,1.138375,292.042202,83.967476,43.43115,252.954167,0.154526
